In [1]:
import pandas as pd
import numpy as np

## Pandas groupby operations

The following functions can help you do simple operations group-wise on a pandas dataset.  This comes in handy when you want to do these operations on an exported .csv that includes many different campaigns, advertisers, lines in one document.  

The functions can be applied to a pandas.groupby object using the following syntax:

        df.groupby(by=column_name).apply(lambda g: divide(g, num_col, denom_col))

In [2]:
# for a group, get the ratio of one column sum to another.  This is useful for finding CVR by strategy
def divide(g, num_col, denom_col):
    return df.loc[g.index, num_col].sum()/df.loc[g.index, denom_col].sum().astype(float) 

# get a weighted average by group
def wavg(g, weight_col):
    return np.average(g, weights = df.loc[g.index, weight_col])

# get win rate by site or by strategy
def pct_won(g):
    return divide(g, 'num_won', 'num_bids')*100

# get average bid, weighted by the number of bids
def w_avg_bid(g):
    return wavg(g, 'num_bids')

# get average payout, weighted by number of won bids
def w_avg_paid(g):
    return wavg(g, 'num_won')

# click-through rate by strategy or by site
def ctr(g):
    return divide(g, 'clicks', 'imps')*100

# total revenue in a group, divided by the # conversions
def ecpa(g):
    return divide(g, 'revenue', 'cons')

def revenue(g):
    return df.loc[g.index, 'eCPM'].unique()[0]*df.loc[g.index, 'num_won'].sum()/1000
    
def margin(g):
    # revenue - total spend divided by revenue
    return 100*(df.loc[g.index, 'revenue'].sum() - df.loc[g.index, 'total_spend'].sum())/df.loc[g.index, 'revenue'].sum().astype(float)

def convert_to_minutes(txt):
    if 'day' in txt:
        split = [item for item in txt.split(' ') if item != '']
        start = int(split[0])*24*60
        end = int(split[1])*24*60
        return str(start) + ' ' + str(end) + ' ' + split[2].replace('day', 'minute')
    return txt



## Read data

In [3]:
df = pd.read_csv('../data/sample_data.csv', sep = ',', header = 0, index_col = 0)
df.head()

,site_id,strategy_id,list_type,line_id,adv_id,adv_vertical,name,goal,price,limit,...,win_rate_site,win_rate_strat,cvr_strat,cvr,line_cvr,hist_zscore,overlap,target_variable,win_rate_site_table,win_rate_strat_table
0,82932,313729,testing,20049,206.0,Travel,Nicole,0.0,3.95,10000.0,...,0.423778,0.111431,0.0,0.001197,0.0,2.708366,0.001066,0,0.450094,0.249479
1,90474,313729,testing,20049,206.0,Travel,Nicole,0.0,3.95,10000.0,...,0.163010,0.111431,0.0,0.001239,0.0,1.188635,0.000703,0,0.158050,0.249479
2,92345,313729,testing,20049,206.0,Travel,Nicole,0.0,3.95,10000.0,...,0.318358,0.111431,0.0,0.000729,0.0,1.503285,0.000873,0,0.360591,0.249479
3,92415,313729,testing,20049,206.0,Travel,Nicole,0.0,3.95,10000.0,...,0.133199,0.111431,0.0,0.005894,0.0,35.153628,0.004614,0,0.113717,0.249479
4,92425,313729,testing,20049,206.0,Travel,Nicole,0.0,3.95,10000.0,...,0.379310,0.111431,0.0,0.000000,0.0,-0.091378,0.000344,0,0.019308,0.249479


## Drop DF rows according to a condition

In [4]:
# drop all records that are not associated with Ian or Nicole
Ian_and_Nicole = df.drop(df[ ~ (df['name'].str.contains('Ian') | df['name'].str.contains('Nicole'))].index)
Ian_and_Nicole['name'].unique()

array(['Nicole', 'Ian'], dtype=object)

## Sort a dataframe

In [5]:
df.sort_values(by = ['line_id', 'strategy_id', 'site_id'], ascending = False, axis = 0, inplace=True)
df.reset_index(drop = True, inplace = True)
df.head()

,site_id,strategy_id,list_type,line_id,adv_id,adv_vertical,name,goal,price,limit,...,win_rate_site,win_rate_strat,cvr_strat,cvr,line_cvr,hist_zscore,overlap,target_variable,win_rate_site_table,win_rate_strat_table
0,1114119077,359117,testing,29695,658.0,Automotive,Edward,35.0,3.24,75000.0,...,0.352060,0.146937,0.000235,0.002625,0.000157,16.512526,0.001697,0,0.361192,0.156831
1,1104262560,359117,testing,29695,658.0,Automotive,Edward,35.0,3.24,75000.0,...,NaN,0.146937,0.000235,0.001369,0.000157,12.733262,0.030301,0,0.195153,0.156831
2,1102384171,359117,testing,29695,658.0,Automotive,Edward,35.0,3.24,75000.0,...,0.526733,0.146937,0.000235,0.002047,0.000157,32.661064,0.014129,0,0.499383,0.156831
3,1099671672,359117,testing,29695,658.0,Automotive,Edward,35.0,3.24,75000.0,...,0.065434,0.146937,0.000235,0.001479,0.000157,4.129372,0.003913,0,0.055280,0.156831
4,1097198698,359117,testing,29695,658.0,Automotive,Edward,35.0,3.24,75000.0,...,0.026784,0.146937,0.000235,0.003135,0.000157,91.398970,0.163626,0,0.033759,0.156831


## Fill NA values

With the most commonly occurring value:

In [6]:
df['win_rate_site'].fillna(df['win_rate_site'].value_counts().idxmax(), inplace = True)

## Create a column conditional on the values of another column

First, create quartiles of conversion rate, and add a new column that indicates the CVR quartile.

In [7]:
df['cvr_quartile'] = pd.qcut(df['cvr'], 4, labels = False)

print "the quartiles correspond to the following CVRs: "
print '\n'.join([str(i) + ': ' + x for i,x in enumerate(pd.qcut(sorted(df['cvr']), 4).unique())])

df[['cvr', 'cvr_quartile']].head(10)

the quartiles correspond to the following CVRs: 
0: [0, 0.000543]
1: (0.000543, 0.00266]
2: (0.00266, 0.0152]
3: (0.0152, 1]


,cvr,cvr_quartile
0,0.002625,1
1,0.001369,1
2,0.002047,1
3,0.001479,1
4,0.003135,2
5,0.002036,1
6,0.002533,1
7,0.003578,2
8,0.003246,2
9,0.001891,1


## Map integers to a string-based value:

For this example, we will map the above quartiles to string-based values from "low" to "great".
This can also be useful when wanting to map first to last names, cities to states, or string-to-int.

In [8]:
group_names = {0:'Low', 1:'Okay', 2:'Good', 3:'Great'}
df['cvr_quartile'] = df['cvr_quartile'].map(group_names)
df[['cvr', 'cvr_quartile']].head()

,cvr,cvr_quartile
0,0.002625,Okay
1,0.001369,Okay
2,0.002047,Okay
3,0.001479,Okay
4,0.003135,Good


In [9]:
df.columns

Index([u'site_id', u'strategy_id', u'list_type', u'line_id', u'adv_id',
       u'adv_vertical', u'name', u'goal', u'price', u'limit', u'avg_bid',
       u'max_bid', u'impressions', u'conversions', u'avg_imps_site',
       u'stdev_imps_site', u'win_rate_site', u'win_rate_strat', u'cvr_strat',
       u'cvr', u'line_cvr', u'hist_zscore', u'overlap', u'target_variable',
       u'win_rate_site_table', u'win_rate_strat_table', u'cvr_quartile'],
      dtype='object')

## Apply a dictionary of operations to different columns

First, read in the data

In [46]:
report = pd.read_csv('../data/sample_pandas_groupby_data.csv', sep = ',', header = 0)
report.head()

,strategy_id,num_bids,num_won,price,imps,clicks,cons,eCPM,revenue,total_spend
0,510301,1,1,3.00,1,0,0,4.0,0.00,0.00
1,510292,1,0,NaN,0,0,0,NaN,0.00,0.00
2,510294,1,1,3.00,1,0,0,4.0,0.00,0.00
3,510292,2,2,1.25,2,0,0,4.0,0.01,0.00
4,510299,4,4,6.00,4,0,0,4.0,0.02,0.02


Next, for each row calculate the CTR and eCPA

In [47]:
report['CTR'] = report['clicks']/report['imps'].astype(float)
report['eCPA'] = report['price']/report['cons'].astype(float)*1000

Finally, create a dictionary of functions to apply to each row of interest.  We will then group by strategy_id and apply the function to each group

In [50]:
apply_dict = {'num_bids': sum,'price':np.mean, 'imps':sum, 'eCPA': lambda g: report.ix[g.index]['total_spend'].sum()/report.ix[g.index]['cons'].sum()*1000}

results = report.groupby(by=['strategy_id'], as_index = False).agg(apply_dict)
results.head(10)

,strategy_id,imps,price,num_bids,eCPA
0,510278,78268,0.100000,6697313,278.888889
1,510285,585,1.006667,8861,9.482759
2,510287,511,1.830000,7406,14.642857
3,510290,1984,1.000000,69527,13.304348
4,510292,4082,1.146000,233738,9.586207
5,510294,5141,1.501667,332902,9.446154
6,510297,4770,0.660000,503730,9.230769
7,510299,1696,1.986000,29102,13.076923
8,510301,1527,1.370000,33310,18.409091
9,510304,5028,0.613333,240817,10.085106


#### For more tricks, see pandas creator Wes McKinney's book ["Python for Data Analysis"](http://www3.canisius.edu/~yany/python/Python4DataAnalysis.pdf)